### Introduction
In this notebook I will save the basic/advanced *python* code that I have encountered during my learning phase. I started with a simple OOP class and the goal is to extend it with a diverse syntax and a set of tricks learned through time. The awaited result is a compact and simple yet rich piece of code.


#### Syntax
- __Slots__ : Python uses the dictionaries to manage the instance attributes. It allows you to add more attributes dynamically. but has a certain memory overhead. "slots" optimizes the memory if the class has many objects. (around 20% performance improvement). SLOTS DO NOT WORK WITH MULTIPLE INHERITANCE! https://stackoverflow.com/a/1816511
- __Descriptors__ : are Python objects that implement a method of the descriptor protocol (get, set, delete, set_name). If your descriptor implements just get(), then it’s a non-data descriptor. If it implements set() or delete(), then it’s said to be a data descriptor.
- __Debugging__ : on jupyterlab by clicking on the "bug icon". (top right)
- __Type_Hinting__ : you're giving a hint about what a variable should be to the IDE and to other developers. If you go against the type hints, python won't stop you!
- __Classmethod__ : https://www.programiz.com/python-programming/methods/built-in/classmethod
- __Inheritance__ : Multiple-resolution order https://www.datacamp.com/tutorial/super-multiple-inheritance-diamond-problem
- __Method Resolution Order (MRO)__ : 
If two superclasses have the same method name and the derived class calls that method, Python uses the MRO to search for the right method to call The MRO specifies that methods should be inherited from the leftmost superclass first.
- __Yield__ : When a function containing yield is called, it returns a generator object. The function is not terminated but paused, and it retains its state. The next time the generator's "next" method is called, it resumes execution from where it was paused.,
- __GIL__ : A global interpreter lock (GIL) is a mechanism used to synchronize the execution of threads so that only one native thread (per process) can execute at a time. An interpreter that uses GIL always allows exactly one thread to execute at a time, even if run on a multi-core processor. It is needed in CPython because memory management is not thread-safe.

#### Tools
- **Convert jupyter notebook to python script**:  
jupyter nbconvert --to script src/notebooks/Python_basics.ipynb --output-dir ./src/

- **Generate requirements.txt**:  
pip install pipreqs  
pipreqs /path/to/project



In [1]:
from dataclasses import dataclass
from typing import Callable
import functools
import time

In [2]:
@dataclass #(slots=True) for python > 3.10 | automate creation of __init__ method
class Person:
    name: str
    surname: str
    adress: str


class Dog:
    def __init__(self, name, owner: Person, breed):
        self._name = name # private variables
        self.owner = owner
        self.__breed = breed # name mangling: "__breed" is replaced with "_Dog__breed"
    
    @staticmethod
    def bark():
        print("Wof Wof!")


class Robot:
    _robots = []
    __slots__ = "id", "_name", "owner","year", "color"
    
    def __init__(self, id, name, owner: Person, year, color="Black"):
        try:
            assert id >= 0, "Robot id greater than 0 expected: " + id
            self.id = id
            self.owner = owner
            self._name = name
            self.year = year
            self.color = color
            Robot._robots.append(self)
            
        except AssertionError:
            print("The robot ID must be greater or equal to 0!")
        
    def _get_name(self):
        return self._name
    
    def _set_name(self, name):
        self._name = name
    
    @classmethod
    def constructed_robots(cls):
        return cls._robots
    
    name = property(_get_name, _set_name)


class RobotDog(Robot, Dog):
    def __init__(self, id, name, owner: Person, year, color="Black", breed="Unknown"):
        super().__init__(id, name, owner, year, color) # Robot.__init__(self, id, name, owner, year, color) 
        # class mro: [__main__.RobotDog, __main__.Robot, __main__.Dog, object]
        # push super to change its mro and look one level above Robot, thus calling super __init__ of dog
        super(Robot, self).__init__(name, owner, breed) # Dog.__init__(self, name, owner, breed) 

    # Change what gets printed with "print()" by defining a special instance method called .__str__().
    def __str__(self):
        return f"{self._name} is a {type(self).__name__}"


Robot.constructed_robots = classmethod(Robot.constructed_robots)

#### Callable and functools.partial

In [3]:
# The callable() method returns True if the object passed appears callable.
def make_dog(c: str) -> Callable[[str], Dog]:
    return lambda a,b: Dog(a, b, c)

# create persons
person_0 = Person("Thomas", "Müller", "76131 Karlsruhe")

# create dogs with callable and functools.partial
make_german_shepherd = make_dog("german_shepherd") # approach 1
make_french_bulldog = functools.partial(Dog, breed="french_bulldog") # approach 2
dog_0 = make_german_shepherd("Flash", person_0)
dog_1 = make_french_bulldog("Iggy", person_0)
print(dog_1.__dict__)

# create robots (add "_" for large numbers to increase readability)
robot_0 = Robot(20_120_000, "Spot", person_0, 1995, "Green")
robot_1 = Robot(10_100_000, "Tops", person_0, 1987, "Red")
robot_2 = Robot(20_130_000, "Robocop", person_0, 1987, "Grey")
robot_3 = RobotDog(10_250_000, "Spip", person_0, 1990, "Black", "French bulldog" )

{'_name': 'Iggy', 'owner': Person(name='Thomas', surname='Müller', adress='76131 Karlsruhe'), '_Dog__breed': 'french_bulldog'}


#### Tuple unpacking manipulation and __slots__

In [4]:
# Using "*" and "_"
_, *desc, color = robot_0.__slots__

# "id", "_name", "owner","year", "color"
print("Some of Robot 0 properties: " + str(desc))

Some of Robot 0 properties: ['_name', 'owner', 'year']


#### Lambda function

In [5]:
newer_robot = lambda x, y: x if (x.year > y.year) else y

print("Newest robot is : " + newer_robot(robot_0, robot_1).name)

Newest robot is : Spot


#### Zip(), enumerate() and List comprehension
I learned about "zip" when I worked with Haskell. I tend to forget about this awesome funtion!

In [6]:
robots = Robot.constructed_robots()
num_robots = len(Robot.constructed_robots())

# 2 different ways but same result
print([(num, robot.name) for num, robot in enumerate(robots)]) 
print([(num, robot.name) for num, robot in zip(range(0, num_robots, 1), robots)]) # range(start, stop, step)

[(0, 'Spot'), (1, 'Tops'), (2, 'Robocop'), (3, 'Spip')]
[(0, 'Spot'), (1, 'Tops'), (2, 'Robocop'), (3, 'Spip')]


#### Map(), any() and all()

In [7]:
def paint_robot(rbt: Robot, color):
    rbt.color = color
    return rbt

robots = Robot.constructed_robots()
is_red_painted = [rbt.color == "Red" for rbt in robots]
print("At least one robot is painted in red: " + str(any(is_red_painted)))
print("All robots are painted in red: " + str(all(is_red_painted)) + "\nLets paint them all in red!\n")


list(map(paint_robot, robots, ["Red"]*len(robots)))
is_red_painted = [rbt.color == "Red" for rbt in robots]
print("At least one robot is painted in red: " + str(any(is_red_painted)))
print("All robots are painted in red: " + str(all(is_red_painted)))

At least one robot is painted in red: True
All robots are painted in red: False
Lets paint them all in red!

At least one robot is painted in red: True
All robots are painted in red: True


#### Multiple inheritance, name mangling, static methods and MRO

In [8]:
is_robot = isinstance(robot_3, Robot)
print(is_robot)

is_robot_dog = isinstance(robot_3, RobotDog) 
if (is_robot_dog):
    robot_3.bark() # RobotDog.bark() also works ==> static method

RobotDog.mro()
# robot_3.__breed raises an error because of name mangling convention

True
Wof Wof!


[__main__.RobotDog, __main__.Robot, __main__.Dog, object]

#### Decorator

In [9]:
def executionTime(func):
    def wrapper():
        t_start = time.time()
        func()
        t_end = time.time() - t_start
        print(f"{func.__name__} Took {t_end} seconds")
        
    return wrapper

@executionTime
def car_simulation():
    time.sleep(1.2)

# @executionTime is just an easier way of saying executionTime(car_simulation). It’s how you apply a decorator to a function.
car_simulation()

car_simulation Took 1.2042655944824219 seconds


#### Recursion

In [10]:
def factorial(n):
    # base case
    if n == 1:
        return 1

    # recurse
    return n * factorial(n-1)

factorial(5)

120

#### Generators and lazy recursion (Haskell inspired)

In [11]:
# yield produce a sequence of values over multiple calls.
def nats(n):
    yield n
    yield from nats(n+1)

generator = nats(2)

In [12]:
# Each iteration
next(generator )

2

#### Shallow vs deep copying

In [13]:
import copy

person_1 = Person("Peter", "Müller", "76135 Karlsruhe")
person_2 = copy.copy(person_1)
person_2.name = "Andreas"

print(person_1)
print(person_2)

Person(name='Peter', surname='Müller', adress='76135 Karlsruhe')
Person(name='Andreas', surname='Müller', adress='76135 Karlsruhe')


#### Itertools

In [14]:
# product
from itertools import product
list_a = [1, 2]
list_b = [3, 4]
prod = product(list_a, list_b) # [(1, 3), (1, 4), (2, 3), (2, 4)]
prod = product(list_a, list_b, repeat=2) # [(1, 3, 1, 3), (1, 3, 1, 4), (1, 3, 2, 3), (1, 3, 2, 4), (1, 4, 1, 3), ...

# permutations
from itertools import permutations
list_a = [1, 2, 3]
perm = permutations(list_a, 2) # [(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)]

# combinations
from itertools import combinations, combinations_with_replacement
list_a = [1, 2, 3, 4]
comb = combinations(list_a, 2) # [(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
comb_wr = combinations_with_replacement(list_a, 2) # [(1, 1), (1, 2), (1, 3), (1, 4), (2, 2), (2, 3), (2, 4), (3, 3), (3, 4), (4, 4)]

# accumulate
from itertools import accumulate
import operator
list_a = [1, 2, 3, 4]
acc = accumulate(list_a) # [1, 3, 6, 10]
acc = accumulate(list_a, func=operator.mul) # [1, 2, 6, 24]
#  print(list(acc))

# groupby
from itertools import groupby

def smaller_than_3(x):
    return x < 3

list_a = [1, 2, 3, 4]
group_obj = groupby(list_a, key=smaller_than_3)
for key, value in group_obj:
    print(key, list(value))


True [1, 2]
False [3, 4]


#### Data Structures

In [15]:
# List (Ordered, mutable, allow duplicates)
lst = [1, 2, 3, 4, 5, 6, 'hello']
lst[-1] # refers to the last item
print(lst[::2]) # start:end:step_size
lst_cpy = lst # refers to the same list, use list.copy() instead

# List as Stacks -  LIFO
stack = [3, 4, 5]
stack.append(7)
stack.pop() # 7

# Tuple (Ordered, immutable, allow duplicates)
tuple = (1, 2, 'world')

# Set (Unordered, immutable, no duplicates)
set = {1, 2, 3}

# Dictionary (Unordered, mutable, unique key)
dict = {'name': 'John', 'age': 25}

# Lists as Queues - FIFO 
from collections import deque
queue = deque(["Eric", "John", "Michael"])
queue.append("Terry")
queue.popleft() # 'Eric'

# Heap (default min-heap) - Useful when quick access to the element with the highest/lowest priority is needed.
import heapq
my_heap = [3, 1, 4, 1, 5, 9, 2]
heapq.heapify(my_heap) # Convert a list to a heap (heapify)
# heapq._heapify_max(listForTree) # convert to a maxheap
heapq.heappush(my_heap, 0) # Add an element to the heap
smallest = heapq.heappop(my_heap) # Pop and return the smallest element

# Array
from array import array
my_array = array('i', [1, 2, 3])

# Counter
from collections import Counter
str_test = "aaaabbbbcccc"
counter = Counter(str_test)
print(counter) # Counter({'a': 4, 'b': 4, 'c': 4})
print(counter.most_common(1)) # [('a', 4)] get a list of tuples as output

# namedtuple
from collections import namedtuple
Point = namedtuple('Point', 'x,y')
pt = Point(1, -4) # Point(x=1, y=-4)



[1, 3, 5, 'hello']
Counter({'a': 4, 'b': 4, 'c': 4})
[('a', 4)]


#### Threading
**\+** All threads withing a process share the same memory  
**\+** Starting a thread is faster than strating a process  
**\+** Threads are lighter than processes  
**\+** Great for I/O-bound tasks  
  
**\-** not interruptable/killable  
**\-** Race condition: When two computer program processes, or threads, attempt to access the same resource at the same time and cause unwanted behavior.  
**\-** Threading is limited by GIL: only one thread at a time  

In [16]:
import threading
from threading import Thread, Lock
import os
import time

# global variabel
database_value = 0

def increase_database(lock):
    global database_value
    # print(threading.get_native_id()) # .get_ident()

    with lock:
        # lock.acquire()
        local_copy = database_value
        local_copy += 1
        time.sleep(0.1)
        database_value = local_copy
        # lock.release()

threads = []
num_threads = 5
lock = Lock()

print("Start value: " + str(database_value))
# create processes
for i in range(num_threads):
    t = Thread(target=increase_database, args=(lock,))  # .get_ident()
    threads.append(t)

# start
for t in threads:
    t.start()
    
# join: wait for completion
for t in threads:
    t.join()

print("End value: " + str(database_value))

Start value: 0
End value: 5


#### Multiprocessing
**\+** Takes advantage of multiple CPUs and cores  
**\+** Separate memory space (not shared between processes)  
**\+** Processes are interrupable/killable  
**\+** One GIL for each process

**\-** Heavyweight (more memory)  
**\-** IPC (inter-process communication) is more complicated  
**\-** Starting a process is slower than starting a thread  

In [17]:
# The next program does not work in a cell you need to save it and run with python in a terminal
from multiprocessing import Process 
import os
import time

def square_numbers():
    for i in range(100):
        i * i
        time.sleep(0.1)

processes = []
num_processes = os.cpu_count()

if __name__ == '__main__':
    # create processes
    for i in range(num_processes):
        p = Process(target=square_numbers)
        processes.append(p)
    
    # start
    for p in processes:
        p.start()
        
    # join
    for p in processes:
        p.join()